In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm
import time
import requests
import os

class DownloadPDF:
    """
    A class to download a PDF given its URL.
    
    Attributes:
    pdf_url (str): The URL of the PDF to download
    directory (str): The directory where to save the PDF
    """

    def __init__(self, pdf_url, directory):
        self.pdf_url = pdf_url
        self.directory = directory

    def download(self, filename):
        response = requests.get(self.pdf_url)
        if response.status_code == 200:
            filepath = os.path.join(self.directory, filename)
            with open(filepath, 'wb') as f:
                f.write(response.content)
            print(f"PDF saved at {filepath}")
        else:
            print(f"Failed to download PDF: {response.status_code}")

class PubMedPDFNavigator:
    """
    A class to navigate through a web portal to find the URL of a PDF for a given DOI.
    
    Attributes:
    username (str): Username for web portal login
    password (str): Password for web portal login
    doi (str): DOI of the article to find
    directory (str): Directory to save the PDF
    driver (webdriver.Chrome): Selenium web driver for Chrome
    """
    
    def __init__(self, username, password, doi, directory):
        self.username = username
        self.password = password
        self.doi = doi
        self.directory = directory
        self.driver = webdriver.Chrome()
        if not os.path.exists(self.directory):
            os.makedirs(self.directory)

    def login(self):
        self.driver.get("University_of_Manitoba_Library_Login_URL")
        username_field = self.driver.find_element_by_name("username")
        password_field = self.driver.find_element_by_name("password")
        username_field.send_keys(self.username)
        password_field.send_keys(self.password)
        password_field.send_keys(Keys.RETURN)
        time.sleep(2)

    def navigate_to_article(self):
        self.driver.get("PubMed_URL_Accessed_via_University_of_Manitoba_Library")
        search_box = self.driver.find_element_by_name("search_box_name_here")
        search_box.send_keys(self.doi)
        search_box.send_keys(Keys.RETURN)
        time.sleep(2)

    def locate_pdf_url(self):
        pdf_url_element = self.driver.find_element_by_css_selector("CSS_Selector_for_PDF_Link")
        pdf_url = pdf_url_element.get_attribute("href")
        return pdf_url

    def run(self):
        self.login()
        self.navigate_to_article()
        pdf_url = self.locate_pdf_url()
        
        # Download PDF using DownloadPDF class
        filename = f"{self.doi.replace('/', '_')}.pdf"
        downloader = DownloadPDF(pdf_url, self.directory)
        downloader.download(filename)

        self.driver.quit()

class BulkPDFDownloader:
    """
    A class to bulk download PDFs for a list of DOIs.

    Attributes:
    username (str): Username for web portal login
    password (str): Password for web portal login
    dois (list): List of DOIs to download
    directory (str): Directory to save PDFs
    
    Example usage:
    username = "your_username_here"
    password = "your_password_here"
    dois = ["example_doi_1", "example_doi_2", "example_doi_3"]  # Replace with your list of DOIs
    directory = "./pdfs" # Repalce with your directory to download PDFs into

    bulk_downloader = BulkPDFDownloader(username, password, dois, directory)
    bulk_downloader.run()
    """

    def __init__(self, username, password, dois, directory):
        self.username = username
        self.password = password
        self.dois = dois
        self.directory = directory
        if not os.path.exists(self.directory):
            os.makedirs(self.directory)

    def run(self):
        # Initialize PubMedPDFNavigator and login
        navigator = PubMedPDFNavigator(self.username, self.password, None, self.directory)
        navigator.login()

        # Loop through each DOI to find PDF URL and download
        for doi in tqdm(self.dois):
            navigator.doi = doi
            navigator.navigate_to_article()
            pdf_url = navigator.locate_pdf_url()

            # Download PDF using DownloadPDF class
            filename = f"{doi.replace('/', '_')}.pdf"
            downloader = DownloadPDF(pdf_url, self.directory)
            downloader.download(filename)

            print(f"Downloaded PDF for DOI: {doi}")

        # Close the browser
        navigator.driver.quit()